In [6]:
import pandas as pd
import gzip as gzip
import ast
import json

In [7]:
#se carga el dataset user reviews
archivo = "../dataSet/user_reviews.json.gz"

data = []

#se lee cada linea
with gzip.open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_user_reviews = pd.DataFrame(data)


In [8]:
df_user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [9]:
df_user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [10]:
#1. identificar los registros duplicados que tiene la columna id
duplicated_user = df_user_reviews[df_user_reviews.duplicated('user_id', keep=False)]

#2. #1. mostrar los registros duplicados que tiene la columna id
duplicated_user

,user_id,user_url,reviews
9,76561198156664158,http://steamcommunity.com/profiles/76561198156...,"[{'funny': '', 'posted': 'Posted June 16.', 'l..."
50,Rivtex,http://steamcommunity.com/id/Rivtex,"[{'funny': '', 'posted': 'Posted December 23, ..."
83,76561198094224872,http://steamcommunity.com/profiles/76561198094...,[]
119,DieMadchenschanderin,http://steamcommunity.com/id/DieMadchenschanderin,"[{'funny': '', 'posted': 'Posted August 29, 20..."
147,relesprit,http://steamcommunity.com/id/relesprit,"[{'funny': '', 'posted': 'Posted December 27, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


In [11]:
# Se revisa un usuario de ejemplo
user_id = '76561198156664158'
user_reviews = duplicated_user[duplicated_user['user_id'] == user_id]['reviews']

for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('-' * 40)

love it
----------------------------------------
love it
----------------------------------------


In [12]:
# Eliminar duplicados en la columna 'user_id', manteniendo solo la primera aparición
df_user_reviews = df_user_reviews.drop_duplicates(subset='user_id', keep='first')

In [13]:
# Se observa el tipo de dato que contiene 'review'
df_user_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [14]:
#cada una de las datos que se encuentra que se encuentra en el diccionario de reviews se tranforma en filas
df_user_reviews = df_user_reviews.explode('reviews').reset_index(drop=True)

Transformacion de la columna 'reviews'

la columna 'reviews' se encuentra anidada, que tiene mas de un diccionario, se debe hacer una columna por cada diccionario,

In [15]:
# Se transforma a columnas cada elemento de las lista anidada
df_user_reviews2 = pd.json_normalize(df_user_reviews['reviews'])
df_user_reviews2.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [16]:
#se selecciona las columnas 'review' y 'item_id'
df_user_reviews2 = df_user_reviews2[['review', 'item_id', 'recommend']]

In [17]:
df_user_reviews2

,review,item_id,recommend
0,Simple yet with great replayability. In my opi...,1250,True
1,It's unique and worth a playthrough.,22200,True
2,Great atmosphere. The gunplay can be a bit chu...,43110,True
3,I know what you think when you see this title ...,251610,True
4,For a simple (it's actually not all that simpl...,227300,True
...,...,...,...
58453,a must have classic from steam definitely wort...,70,True
58454,this game is a perfect remake of the original ...,362890,True
58455,had so much fun plaing this and collecting res...,273110,True
58456,:D,730,True


In [18]:
# Se eliminan las filas con valor None
df_user_reviews2 = df_user_reviews2.dropna()
# Se verifica que solo queden el 'user_id' con la cantidad de diccionarios que le corresponde


In [19]:
print(df_user_reviews2.columns)

Index(['review', 'item_id', 'recommend'], dtype='object')


In [20]:
# se importa la libreria nltk para hacerle analisis de sentimiento a la review
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
analisis= SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to C:\Users\BORDADO Y
[nltk_data]     ESTAMPADO\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [21]:
#se crea una funcion que analisa el sentimiento
def AnalisisSent(text):
    senti= analisis.polarity_scores(text)
    senti= senti['compound']
    return  senti

In [22]:
# se crea una nueva columna que contiene el analisis de sentimiento
df_user_reviews2['sentimentanalysis'] = df_user_reviews2['review'].apply(AnalisisSent)

C:\Users\BORDADO Y ESTAMPADO\AppData\Local\Temp\ipykernel_16888\2964428939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_reviews2['sentimentanalysis'] = df_user_reviews2['review'].apply(AnalisisSent)


In [23]:
#se muestra el dataset nuevo con la columna que creamos anteriormente
df_user_reviews2

,review,item_id,recommend,sentimentanalysis
0,Simple yet with great replayability. In my opi...,1250,True,0.8481
1,It's unique and worth a playthrough.,22200,True,0.2263
2,Great atmosphere. The gunplay can be a bit chu...,43110,True,0.9117
3,I know what you think when you see this title ...,251610,True,0.9566
4,For a simple (it's actually not all that simpl...,227300,True,0.9708
...,...,...,...,...
58453,a must have classic from steam definitely wort...,70,True,0.5574
58454,this game is a perfect remake of the original ...,362890,True,0.9786
58455,had so much fun plaing this and collecting res...,273110,True,0.7827
58456,:D,730,True,0.5106


podemos ver que el sentimiento de los usuarios  a cerca de la mayoria de juegos es bueno o positivo ya que se encuentrar por encima de el cero y algunos se acercan a 1

In [21]:
#se elimina la columna revies por que no es necesaria
#df_user_reviews2 = df_user_reviews2.drop('review', axis=1)

In [25]:
df_user_reviews2.dtypes

review                object
item_id               object
recommend             object
sentimentanalysis    float64
dtype: object

Se guarda el conjunto de datos transformado como `user_review_clean`. 

In [26]:
file_user_clean = '../dataSet/user_review_clean.parquet'
df_user_reviews2.to_parquet(file_user_clean, index=False)
print(f'Se guardó el archivo {file_user_clean}')

Se guardó el archivo ../dataSet/user_review_clean.parquet
